In [1]:
from snv import *
import pandas as pd

# File path
file_path = '/Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/multi_clipp_simulation_data/simulation_data_cluster_5_region_1_read_depth_100_replica_2.tsv'

# Read the data into a DataFrame
df = pd.read_csv(file_path, sep='\t')
rho = 0.8
omega = 1
n = df.shape[0]
m = 1
gamma = 0.01
control_large = 5
precision = 0.01
max_iteration = 20
sets = {i for i in range(n)}
combinations_2 = list(itertools.combinations(sets, 2))
pairs_mapping = {combination: index for index, combination in enumerate(combinations_2)}
pairs_mapping_inverse = {index: combination for index, combination in enumerate(combinations_2)}

# Get matrices
read_mat = get_read_mat(df)
total_read_mat = get_total_read_mat(df)
c_mat = get_c_mat(df)
bb_mat = get_b_mat(df)
tumor_cn_mat = get_tumor_cn_mat(df)
linearApprox = get_linear_approximation(c_mat)
# Initialize variables
#12/12/2024
phi_hat = (read_mat / (total_read_mat * c_mat))
scale_parameter = np.max([1, np.max(phi_hat)])
phi_hat = phi_hat / scale_parameter
phi_hat[phi_hat > sigmoid(control_large)] = sigmoid(control_large)
phi_hat[phi_hat < sigmoid(-control_large)] = sigmoid(-control_large)
p = inverse_sigmoid(phi_hat)
p[p > control_large] = control_large
p[p < -control_large] = -control_large
p = p.reshape([n * m])
v = np.zeros([len(combinations_2) * m])
for i in range(len(combinations_2)):
    pair = combinations_2[i]
    index_v = pairs_mapping[pair]
    start_v = index_v * m
    end_v = (index_v + 1) * m
    l1, l2 = pairs_mapping_inverse[index_v]
    a_mat = a_mat_generator(l1, l2, n, m)
    v[start_v: end_v] = matmul_by_torch(a_mat, p)
    
y = np.ones([len(combinations_2) * m])
omega = np.ones([len(combinations_2)])
k = 0

control_large = 5
wcut = np.array(linearApprox[0])
coef = np.array(linearApprox[1])
temp = 100
# ADMM
while k < max_iteration and precision < temp:
    p = update_p(p, v, y, n, m, read_mat, total_read_mat, bb_mat, tumor_cn_mat, coef, wcut, combinations_2, pairs_mapping, rho, control_large)
    temp = 0
    for i in range(len(combinations_2)):
        pair = combinations_2[i]
        index_v = pairs_mapping[pair]
        start_v = index_v * m
        end_v = (index_v + 1) * m
        v[start_v: end_v] = update_v_SCAD(index_v, pairs_mapping_inverse, p, y, n, m, rho, omega[i], gamma)                
        y[start_v: end_v] = update_y(y[start_v: end_v], v[start_v: end_v], i, pairs_mapping_inverse, p, n, m, rho)
        l1, l2 = pairs_mapping_inverse[index_v]
        a_mat = a_mat_generator(l1, l2, n, m)
        temp = max(temp, np.linalg.norm(matmul_by_torch(a_mat, p) - v[start_v: end_v]))
    rho = 1.02 * rho
    k = k + 1
    print('\r', k, ',', temp, end="")
        
diff = np.zeros((n, n))
class_label = -np.ones(n)
class_label[0] = 0
group_size = [1]
labl = 1
least_mut = 25
for i in range(1, n):
    for j in range(i):
        index_v = pairs_mapping[(j, i)]
        start_v = index_v * m
        end_v = (index_v + 1) * m
        diff[j, i] = np.linalg.norm(v[start_v: end_v]) if np.linalg.norm(v[start_v: end_v]) > 0.05 else 0
        diff[i, j] = diff[j, i]
for i in range(1, n):
    for j in range(i):
        if diff[j, i] == 0:
            class_label[i] = class_label[j]
            group_size[int(class_label[j])] += 1
            break
    if class_label[i] == -1:
        class_label[i] = labl
        labl += 1
        group_size.append(1)


 12 , 0.008943276705263514

In [2]:
# quality control
tmp_size = np.min(np.array(group_size)[np.array(group_size) > 0])
tmp_grp = np.where(group_size == tmp_size)
refine = False
if tmp_size < least_mut:
    refine = True
while refine:
    refine = False
    tmp_col = np.where(class_label == tmp_grp[0][0])[0]
    for i in range(len(tmp_col)):
        if tmp_col[i] != 0 and tmp_col[i] != n - 1:
            tmp_diff = np.abs(np.append(np.append(diff[0:tmp_col[i], tmp_col[i]].T.ravel(), 100),
                                        diff[tmp_col[i], (tmp_col[i] + 1):n].ravel()))
            tmp_diff[tmp_col] += 100
            diff[0:tmp_col[i], tmp_col[i]] = tmp_diff[0:tmp_col[i]]
            diff[tmp_col[i], (tmp_col[i] + 1):n] = tmp_diff[(tmp_col[i] + 1):n]
        elif tmp_col[i] == 0:
            tmp_diff = np.append(100, diff[0, 1:n])
            tmp_diff[tmp_col] += 100
            diff[0, 1:n] = tmp_diff[1:n]
        else:
            tmp_diff = np.append(diff[0:(n - 1), n - 1], 100)
            tmp_diff[tmp_col] += 100
            diff[0:(n - 1), n - 1] = tmp_diff[0:(n - 1)]
        ind = tmp_diff.argmin()
        group_size[class_label.astype(np.int64, copy=False)[tmp_col[i]]] -= 1
        class_label[tmp_col[i]] = class_label[ind]
        group_size[class_label.astype(np.int64, copy=False)[tmp_col[i]]] += 1
    tmp_size = np.min(np.array(group_size)[np.array(group_size) > 0])
    tmp_grp = np.where(group_size == tmp_size)
    refine = False
    if tmp_size < least_mut:
        refine = True
labels = np.unique(class_label)

phi_out = np.zeros((len(labels), m))


In [3]:
for i in range(len(labels)):
    ind = np.where(class_label == labels[i])[0]
    class_label[ind] = i
    phi_out[i, :] = np.sum(phi_hat[ind,: ] * total_read_mat[ind,: ]) / np.sum(total_read_mat[ind, :])
    

In [ ]:
if len(labels) > 1:
    phi_norm = np.linalg.norm(phi_out, axis=1)
    sort_phi = np.sort(phi_norm)
    indices = [np.where(phi_norm == element)[0][0] for element in sort_phi]
    phi_diff = sort_phi[1:] - sort_phi[:-1]
    min_val = phi_diff.min()
    min_ind = phi_diff.argmin()
    while min_val < 0.01:
        combine_ind = np.where(phi_out == sort_phi[indices[min_ind]])[0]
        combine_to_ind = np.where(phi_out == sort_phi[indices[min_ind] + 1])[0]
        class_label[class_label == combine_ind] = combine_to_ind
        labels = np.unique(class_label)
        phi_out = np.zeros(len(labels))
        for i in range(len(labels)):
            ind = np.where(class_label == labels[i])[0]
            class_label[ind] = i
            phi_out[i, :] = np.sum(phi_hat[ind, :] * total_read_mat[ind, :]) / np.sum(total_read_mat[ind, :])
        if len(labels) == 1:
            break
        else:
            phi_norm = np.linalg.norm(phi_out, axis=1)
            sort_phi = np.sort(phi_norm)
            indices = [np.where(phi_norm == element)[0][0] for element in sort_phi]
            phi_diff = sort_phi[1:] - sort_phi[:-1]
            min_val = phi_diff.min()
            min_ind = phi_diff.argmin()
phi_res = np.zeros((n, m))
for lab in range(len(phi_out)):
    phi_res[class_label == lab, :] = phi_out[lab, :]

purity = get_purity_mat(df)[0,0]